In [35]:
# Package that ensures a programatically interaction with operating system folder hierarchy.
from os import listdir

# Package used for clone a dictionary.
from copy import deepcopy

# Functions intended to extract some statistical parameters.
from numpy import max, std, average, sum, absolute

# With the following import we will be able to extract the linear regression parameters after 
# fitting experimental points to the model.
from scipy.stats import linregress

# biosignalsnotebooks own package that supports some functionalities used on the Jupyter Notebooks.
import biosignalsnotebooks as bsnb

# Python package that contains functions specialized on "Machine Learning" tasks.
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV, RFE
from sklearn.preprocessing import normalize

from json import loads, dump

# Package containing a diversified set of function for statistical processing and also provide support to array operations.
from numpy import max, array

# Python package that contains functions specialized on "Machine Learning" tasks.
from sklearn.neighbors import KNeighborsClassifier

# Python package that contains functions specialised on "Machine Learning" tasks.
from sklearn.model_selection import cross_val_score, LeaveOneOut

import numpy as np

import os

In [2]:
# Transposition of data from signal files to a Python dictionary.
relative_path = "/home/ferdinand/realworld_interact_systems"
data_folder = "/home/ferdinand/realworld_interact_systems/data"

# List of files (each file is a training example).
list_examples = listdir(data_folder)

In [3]:
# Initialization of dictionary.
signal_dict = {}

# Scrolling through each entry in the list.
for example in list_examples:
    if ".txt" in example: # Read only .txt files.
        # Get the class to which the training example under analysis belong.
        example_class = example.split("_")[0]

        # Get the trial number of the training example under analysis.
        example_trial = example.split("_")[1].split(".")[0]

        # Creation of a new "class" entry if it does not exist.
        if example_class not in signal_dict.keys():
            signal_dict[example_class] = {}

        # Load data.
        complete_data = bsnb.load(data_folder + "/" + example)

        # Store data in the dictionary.
        signal_dict[example_class][example_trial] = complete_data

In [4]:
# Channels (CH1 Flexor digitorum superficialis | CH2 Aductor policis | CH3 Accelerometer axis Z).
emg_flexor = "CH2"
emg_adductor = "CH3"
acc_z = "CH4"

In [5]:
# Clone "signal_dict".
features_dict = deepcopy(signal_dict)

# Navigate through "signal_dict" hierarchy.
list_classes = signal_dict.keys()
for class_i in list_classes:
    list_trials = signal_dict[class_i].keys()
    for trial in list_trials:
        # Initialise "features_dict" entry content.
        features_dict[class_i][trial] = []
        
        for chn in [emg_flexor, emg_adductor, acc_z]:
            # Temporary storage of signal inside a reusable variable.
            signal = signal_dict[class_i][trial][chn]
            
            # Start the feature extraction procedure accordingly to the channel under analysis.
            if chn == emg_flexor or chn == emg_adductor: # EMG Features.
                # Converted signal (taking into consideration that our device is a "biosignalsplux", the resolution is
                # equal to 16 bits and the output unit should be in "mV").
                signal = bsnb.raw_to_phy("EMG", device="biosignalsplux", raw_signal=signal, resolution=16, option="mV")
                
                # Standard Deviation.
                features_dict[class_i][trial] += [std(signal)]
                # Maximum Value.
                features_dict[class_i][trial] += [max(signal)]
                # Zero-Crossing Rate.
                features_dict[class_i][trial] += [sum([1 for i in range(1, len(signal)) 
                                                       if signal[i]*signal[i-1] <= 0]) / (len(signal) - 1)]
                # Standard Deviation of the absolute signal.
                features_dict[class_i][trial] += [std(absolute(signal))]
            else: # ACC Features.
                # Converted signal (taking into consideration that our device is a "biosignalsplux", the resolution is
                # equal to 16 bits and the output unit should be in "g").
                signal = bsnb.raw_to_phy("ACC", device="biosignalsplux", raw_signal=signal, resolution=16, option="g")
                
                # Average value.
                features_dict[class_i][trial] += [average(signal)]
                # Standard Deviation.
                features_dict[class_i][trial] += [std(signal)]
                # Maximum Value.
                features_dict[class_i][trial] += [max(signal)]
                # Zero-Crossing Rate.
                features_dict[class_i][trial] += [sum([1 for i in range(1, len(signal)) 
                                                       if signal[i]*signal[i-1] <= 0]) / (len(signal) - 1)]
                # Slope of the regression curve.
                x_axis = range(0, len(signal))
                features_dict[class_i][trial] += [linregress(x_axis, signal)[0]]

In [6]:
# Package dedicated to the manipulation of json files.
from json import dump

filename = "classification_game_features.json"

# Generation of .json file in our previously mentioned "relative_path".
# [Generation of new file]
with open(relative_path + "/features/" + filename, 'w') as file:
    dump(features_dict, file)

# Load of data inside file storing it inside a Python dictionary.
with open(relative_path + "/features/" + filename) as file:
    features_dict = loads(file.read())

In [7]:
from sty import fg, rs
print(fg(98,195,238) + "\033[1mDict Keys\033[0m" + fg.rs + " define the class number")
print(fg(232,77,14) + "\033[1mDict Sub-Keys\033[0m" + fg.rs + " define the trial number\n")
print(features_dict)

Dict Keys define the class number
Dict Sub-Keys define the trial number

{'3': {'5': [0.0009411571726303267, -1.469696044921875, 0.0, 0.0009411571726303267, 0.0011068800158332357, -1.4692840576171875, 0.0, 0.0011068800158332357, -6.498812007407408, 5.7436018676605574e-05, -6.49859, 0.0, 1.5383753004301147e-09], '2': [0.0012061097236280257, -1.4590301513671875, 0.0, 0.0012061097236280257, 0.0012277958039775992, -1.4646148681640625, 0.0, 0.0012277958039775992, -6.498806158558561, 6.50968792493583e-05, -6.49859, 0.0, 7.716544844758052e-10], '4': [0.001252298795212228, -1.4642486572265625, 0.0, 0.001252298795212228, 0.0016688729895145695, -1.45330810546875, 0.0, 0.0016688729895145695, -6.498810946296297, 5.80849703673147e-05, -6.49859, 0.0, -1.5039666725178493e-10], '3': [0.0010568122871173681, -1.4647064208984375, 0.0, 0.0010568122871173681, 0.001355744536654651, -1.4626007080078125, 0.0, 0.001355744536654651, -6.498808081481483, 5.785186086208738e-05, -6.49859, 0.0, 7.734926343577733e-10

In [8]:
# Initialisation of a list containing our training data and another list containing the labels of each training example.
features_list = []
class_training_examples = []

# Access each feature list inside dictionary.
list_classes = features_dict.keys()
for class_i in list_classes:
    list_trials = features_dict[class_i].keys()
    for trial in list_trials:
        # Storage of the class label.
        class_training_examples += [int(class_i)]
        features_list += [features_dict[class_i][trial]]

print(fg(232,77,14) + "\033[1m[Number of list entries;Number of sub-list entries]:\033[0m" + fg.rs + " [" + str(len(features_list)) + "; " + str(len(features_list[0])) + "]" + u'\u2713')
print(fg(253,196,0) + "\033[1mClass of each training example:\033[0m" + fg.rs)
print(class_training_examples)
print(fg(98,195,238) + "\033[1mFeatures List:\033[0m" + fg.rs)
print(features_list)


[Number of list entries;Number of sub-list entries]: [20; 13]✓
Class of each training example:
[3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2]
Features List:
[[0.0009411571726303267, -1.469696044921875, 0.0, 0.0009411571726303267, 0.0011068800158332357, -1.4692840576171875, 0.0, 0.0011068800158332357, -6.498812007407408, 5.7436018676605574e-05, -6.49859, 0.0, 1.5383753004301147e-09], [0.0012061097236280257, -1.4590301513671875, 0.0, 0.0012061097236280257, 0.0012277958039775992, -1.4646148681640625, 0.0, 0.0012277958039775992, -6.498806158558561, 6.50968792493583e-05, -6.49859, 0.0, 7.716544844758052e-10], [0.001252298795212228, -1.4642486572265625, 0.0, 0.001252298795212228, 0.0016688729895145695, -1.45330810546875, 0.0, 0.0016688729895145695, -6.498810946296297, 5.80849703673147e-05, -6.49859, 0.0, -1.5039666725178493e-10], [0.0010568122871173681, -1.4647064208984375, 0.0, 0.0010568122871173681, 0.001355744536654651, -1.4626007080078125, 0.0, 0.001355744536654651, -6.4988

In [9]:
features_list = normalize(features_list, axis=0, norm="max") # axis=0 specifies that each feature is normalised independently from the others 
                                                             # and norm="max" defines that the normalization reference value will be the feature maximum value.

In [10]:
# Creation of a "Support Vector Classifier" supposing that  our classes are linearly separable.
svc = SVC(kernel="linear")
rfecv = RFECV(estimator=svc, step=1, cv=StratifiedKFold(5), scoring='accuracy')
# Fit data to the model.
selector = rfecv.fit(features_list, class_training_examples)
# Get list of average score of the virtual classifier
# avg_scores = rfecv.grid_scores_
avg_scores = rfecv.cv_results_['mean_test_score']
print(avg_scores)
max_score = max(avg_scores)
print(fg(98,195,238) + "\033[1mMaximum Average Score:\033[0m " + fg.rs + str(max_score))
for nbr_features in range(0, len(avg_scores)):
    if avg_scores[nbr_features] == max_score:
        optimal_nbr_features = nbr_features + 1
        break
print(fg(98,195,238) + "\033[1mOptimal Number of Features:\033[0m " + fg.rs + str(optimal_nbr_features))
bsnb.plot([range(1, len(rfecv.cv_results_['mean_test_score']) + 1)], [avg_scores], 
          y_axis_label="Cross validation score (nb of correct classifications)", x_axis_label="Number of features selected")

[0.7 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9 0.9]
Maximum Average Score: 0.9
Optimal Number of Features: 2


In [ ]:
rfe = RFE(estimator=svc, step=1, n_features_to_select=optimal_nbr_features)

# Fit data to the model.
final_selector = rfe.fit(features_list, class_training_examples)

# Acception/Rejection Label attributed to each feature.
acception_labels = final_selector.support_

############### from webseite ###############
acception_labels_8 = [True, False, True, True, True, False, False, True, False, True, True, False, True]

print(fg(98,195,238) + "\033[1mRelevant Features (True):\033[0m " + fg.rs)
print(acception_labels)

Relevant Features (True): 
[False False False  True False False False  True False False False False
 False]


In [12]:
# Access each training example and exclude meaningless entries.
final_features_list = []
for example_nbr in range(0, len(features_list)):
    final_features_list += [list(array(features_list[example_nbr])[array(acception_labels_8)])]

In [13]:
filename_final = "classification_game_features_final.json"

# Generation of .json file in our previously mentioned "relative_path".
# [Generation of new file]
with open(relative_path + "/features/" + filename_final, 'w') as file:
    dump({"features_list_final": final_features_list, "class_labels": class_training_examples}, file)

# Load of data inside file storing it inside a Python dictionary.
with open(relative_path + "/features/" + filename_final) as file:
    features_dict = loads(file.read())

from sty import fg, rs
print(fg(98,195,238) + "\033[1mDict Keys\033[0m" + fg.rs + " divides training example in its features and class label")
print(features_dict)

Dict Keys divides training example in its features and class label
{'features_list_final': [[0.26449552396553716, 0.0, 0.26449552396553716, 0.6632500033182258, 0.6632500033182258, 0.8459189411599269, -1.0, 0.8242184310155207], [0.33895573724350375, 0.0, 0.33895573724350375, 0.7357035626388393, 0.7357035626388393, 0.9587482634805633, -1.0, 0.4134308762646644], [0.35193635625743747, 0.0, 0.35193635625743747, 1.0, 1.0, 0.8554767158754681, -1.0, -0.08057832511843604], [0.2969983417680634, 0.0, 0.2969983417680634, 0.8123713099634986, 0.8123713099634986, 0.8520434739763995, -1.0, 0.41441570552658996], [0.3258101218524577, 0.0, 0.3258101218524577, 0.9286055028786828, 0.9286055028786828, 0.9326431452091057, -1.0, 0.4946048017922442], [0.34621003900777786, 0.0, 0.34621003900777786, 0.5478820314900659, 0.5478820314900659, 0.859997459443254, -1.0, 0.3646367052849735], [0.3377773710413215, 0.0, 0.3377773710413215, 0.46675260668520396, 0.46675260668520396, 0.8804300502587237, -1.0, -0.0026319055873

KNN

In [14]:
training_examples = features_dict["features_list_final"]
class_training_examples = features_dict["class_labels"]

# k-Nearest Neighbour object initialisation.
knn_classifier = KNeighborsClassifier()

knn_classifier.fit(training_examples, class_training_examples)

print("Training data shape:", np.array(training_examples).shape)

Training data shape: (20, 8)


In [15]:
import numpy as np
from numpy import array

from bokeh.layouts import layout
from bokeh.models import CustomJS, Slider, Select, ColumnDataSource, WidgetBox
from bokeh.plotting import figure, show

tools = 'pan'
features_identifiers = ["std_emg_flexor", "zcr_emg_flexor", "std_abs_emg_flexor", "std_emg_adductor", "std_abs_emg_adductor", "std_acc_z", "max_acc_z", "m_acc_z"]

def slider():
    dict_features = {}
    for feature_nbr in range(0, len(training_examples[0])):
        values_feature = array(training_examples)[:, feature_nbr]
        
        # Fill of dict.
        for class_of_example in range(0, len(class_training_examples)):
            current_keys = list(dict_features.keys())
            if class_training_examples[class_of_example] not in current_keys:
                dict_features[class_training_examples[class_of_example]] = {}
            
            current_sub_keys = list(dict_features[class_training_examples[class_of_example]].keys())
            if features_identifiers[feature_nbr] not in current_sub_keys:
                dict_features[class_training_examples[class_of_example]][features_identifiers[feature_nbr]] = []
            
            dict_features[class_training_examples[class_of_example]][features_identifiers[feature_nbr]] += [values_feature[class_of_example]]
            
            # Add of two additional keys that will store the data currently being ploted.
            if feature_nbr == 0:
                if "x" not in current_sub_keys:
                    dict_features[class_training_examples[class_of_example]]["x"] = []
                dict_features[class_training_examples[class_of_example]]["x"] += [values_feature[class_of_example]]
            elif feature_nbr == 1:
                if "y" not in current_sub_keys:
                    dict_features[class_training_examples[class_of_example]]["y"] = []
                dict_features[class_training_examples[class_of_example]]["y"] += [values_feature[class_of_example]]
        
    source_class_0 = ColumnDataSource(data=dict_features[0])
    source_class_1 = ColumnDataSource(data=dict_features[1])
    source_class_2 = ColumnDataSource(data=dict_features[2])
    source_class_3 = ColumnDataSource(data=dict_features[3])

    plot = figure(x_range=(-1.5, 1.5), y_range=(-1.5, 1.5), tools='', toolbar_location=None, title="Pairing Classification Dimensions")
    bsnb.opensignals_style([plot])
    
    # Define different colours for points of each class.
    # [Class 0]
    plot.circle('x', 'y', source=source_class_0, line_width=3, line_alpha=0.6, color="red")
    # [Class 1]
    plot.circle('x', 'y', source=source_class_1, line_width=3, line_alpha=0.6, color="green")
    # [Class 2]
    plot.circle('x', 'y', source=source_class_2, line_width=3, line_alpha=0.6, color="orange")
    # [Class 3]
    plot.circle('x', 'y', source=source_class_3, line_width=3, line_alpha=0.6, color="blue")

    callback = CustomJS(args=dict(source=[source_class_0, source_class_1, source_class_2, source_class_3]), code="""
        // Each class has an independent data structure.
        var data_0 = source[0].data;
        var data_1 = source[1].data;
        var data_2 = source[2].data;
        var data_3 = source[3].data;
        
        // Selected values in the interface.
        var feature_identifier_x = x_feature.value;
        var feature_identifier_y = y_feature.value;
        console.log("x_feature: " + feature_identifier_x);
        console.log("y_feature: " + feature_identifier_y);
        
        // Update of values.
        var x_0 = data_0["x"];
        var y_0 = data_0["y"];
        for (var i = 0; i < x_0.length; i++) {
            x_0[i] = data_0[feature_identifier_x][i];
            y_0[i] = data_0[feature_identifier_y][i];
        }
        
        var x_1 = data_1["x"];
        var y_1 = data_1["y"];
        for (var i = 0; i < x_1.length; i++) {
            x_1[i] = data_1[feature_identifier_x][i];
            y_1[i] = data_1[feature_identifier_y][i];
        }
        
        var x_2 = data_2["x"];
        var y_2 = data_2["y"];
        for (var i = 0; i < x_2.length; i++) {
            x_2[i] = data_2[feature_identifier_x][i];
            y_2[i] = data_2[feature_identifier_y][i];
        }
        
        var x_3 = data_3["x"];
        var y_3 = data_3["y"];
        for (var i = 0; i < x_3.length; i++) {
            x_3[i] = data_3[feature_identifier_x][i];
            y_3[i] = data_3[feature_identifier_y][i];
        }
        
        // Communicate update.
        source[0].change.emit();
        source[1].change.emit();
        source[2].change.emit();
        source[3].change.emit();
    """)

    # x_feature_select = Select(title="Select the Feature of Axis x:", value="std_emg_flexor", options=features_identifiers, callback=callback)
    # callback.args["x_feature"] = x_feature_select
    
    # y_feature_select = Select(title="Select the Feature of Axis y:", value="zcr_emg_flexor", options=features_identifiers, callback=callback)
    # callback.args["y_feature"] = y_feature_select

    x_feature_select = Select(
        title="Select the Feature of Axis x:",
        value="std_emg_flexor",
        options=features_identifiers
    )
    y_feature_select = Select(
        title="Select the Feature of Axis y:",
        value="zcr_emg_flexor",
        options=features_identifiers
    )

    # Register your callback properly using js_on_change()
    callback.args["x_feature"] = x_feature_select
    callback.args["y_feature"] = y_feature_select

    x_feature_select.js_on_change("value", callback)
    y_feature_select.js_on_change("value", callback)

    widgets = WidgetBox(x_feature_select, y_feature_select)
    return [widgets, plot]

l = layout([slider(),], sizing_mode='scale_width')

show(l)

In [16]:
# A list with 8 arbitrary entries.
# test_examples_features = [0.65, 0.51]
test_examples_features = [0.65, 0.51, 0.70, 0.10, 0.20, 0.17, 0.23, 0.88]

# Classification.
print("Returned Class: ")
print(knn_classifier.predict([test_examples_features]))

# Probability of Accuracy.
print("Probability of each class:")
print(knn_classifier.predict_proba([test_examples_features]))

Returned Class: 
[2]
Probability of each class:
[[0.4 0.  0.6 0. ]]


In [17]:
# Load of data inside file, storing it inside a Python dictionary.
with open(relative_path + "/features/" + filename_final) as file:
    features_class_dict = loads(file.read())

features_class_dict.keys()

features_list = features_class_dict["features_list_final"]
class_training_examples = features_class_dict["class_labels"]
print(len(features_list[0]))

# Renaming original variable.
training_examples_a = features_list
print(len(features_list))

8
20


In [18]:
# k-Nearest Neighbour object initialisation.
knn_classifier_a = KNeighborsClassifier()

# Fit model to data.
knn_classifier_a.fit(training_examples_a, class_training_examples) 

,n_neighbors,5
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [19]:
leave_one_out_score_a = cross_val_score(knn_classifier_a, training_examples_a, class_training_examples, scoring="accuracy", cv=LeaveOneOut())

# Average accuracy of classifier.
mean_l1o_score_a = leave_one_out_score_a.mean()

# Standard Deviation of the previous estimate.
std_l1o_score_a = leave_one_out_score_a.std()

In [20]:
from sty import fg, rs
print(fg(232,77,14) + "\033[1mAverage Accuracy of Classifier:\033[0m" + fg.rs)
print(str(mean_l1o_score_a * 100) + " %")

print(fg(98,195,238) + "\033[1mStandard Deviation:\033[0m" + fg.rs)
print("+-" + str(round(std_l1o_score_a, 1) * 100) + " %")

Average Accuracy of Classifier:
85.0 %
Standard Deviation:
+-40.0 %


In [36]:
import numpy as np
from scipy.stats import linregress

# --- Define which 8 features to keep (Set A) ---
# Order: [σ_emg_flexor, zcr_emg_flexor, σ_abs_emg_flexor,
#         σ_emg_adductor, σ_abs_emg_adductor,
#         σ_acc_z, max_acc_z, m_acc_z]
acception_labels_8 = np.array([
    True, True, True, True, True, True, True, True,
    False, False, False, False, False
])


# --- Feature extraction for one recording ---
def extract_features(signal):
    """
    Extract the same 13 features as in training
    from one raw signal recording (.txt).
    Channels:
        CH1 = EMG flexor
        CH2 = EMG adductor
        CH3 = Accelerometer Z
    """
    signal = np.array(signal)

    # Ensure correct shape (n_samples, n_channels)
    if signal.ndim == 1:
        signal = signal.reshape(-1, 1)

    # Split channels
    emg_flexor = signal[:, 0]
    emg_adductor = signal[:, 1]
    acc_z = signal[:, 2]

    # --- EMG Flexor ---
    sigma_emg_flexor = np.std(emg_flexor)
    zcr_emg_flexor = np.sum(np.diff(np.sign(emg_flexor)) != 0) / len(emg_flexor)
    sigma_abs_emg_flexor = np.std(np.abs(emg_flexor))

    # --- EMG Adductor ---
    sigma_emg_adductor = np.std(emg_adductor)
    sigma_abs_emg_adductor = np.std(np.abs(emg_adductor))

    # --- Accelerometer Z ---
    sigma_acc_z = np.std(acc_z)
    max_acc_z = np.max(acc_z)
    m_acc_z = np.mean(acc_z)

    # --- Optional extras (to fill full 13-feature vector) ---
    mean_all = np.mean(signal)
    std_all = np.std(signal)
    min_acc_z = np.min(acc_z)
    slope_acc_z = linregress(np.arange(len(acc_z)), acc_z)[0]
    zcr_acc_z = np.sum(np.diff(np.sign(acc_z)) != 0) / len(acc_z)

    # --- Combine all into one 13-feature vector ---
    features_13 = [
        sigma_emg_flexor, zcr_emg_flexor, sigma_abs_emg_flexor,
        sigma_emg_adductor, sigma_abs_emg_adductor,
        sigma_acc_z, max_acc_z, m_acc_z,
        mean_all, std_all, min_acc_z, slope_acc_z, zcr_acc_z
    ]

    return features_13


# --- Classification function ---
def classify_new_recording(filepath, classifier, feature_mask=acception_labels_8):
    """
    Reads one file, extracts features, selects the 8 important ones,
    and classifies with a trained model.
    """
    # Load raw signal
    signal = np.loadtxt(filepath)

    # Extract all 13 features
    full_features = extract_features(signal)

    # Select only the 8 relevant features
    reduced_features = np.array(full_features)[feature_mask]

    # Predict
    pred = classifier.predict([reduced_features])[0]
    return pred



# --- Classify all recordings in a folder ---
def classify_all_in_folder(data_folder, classifier, feature_mask=acception_labels_8):
    """Classify all .txt files in the given folder."""
    results = []
    files = sorted([f for f in os.listdir(data_folder) if f.endswith(".txt")])

    for fname in files:
        fpath = os.path.join(data_folder, fname)
        try:
            pred = classify_new_recording(fpath, classifier, feature_mask)
            results.append((fname, pred))
            print(f"{fname:25s} → Predicted class: {pred}")
        except Exception as e:
            print(f"⚠️ Error processing {fname}: {e}")

    return results


# --- Example usage ---
data_folder = "/home/ferdinand/realworld_interact_systems/data"
results = classify_all_in_folder(data_folder, knn_classifier_a)



# --- Example usage ---
pred = classify_new_recording(
    "/home/ferdinand/realworld_interact_systems/data/0_1.txt",
    knn_classifier_a
)
print("Predicted class:", pred)


0_1.txt                   → Predicted class: 2
0_2.txt                   → Predicted class: 2
0_3.txt                   → Predicted class: 2
0_4.txt                   → Predicted class: 2
0_5.txt                   → Predicted class: 2
1_1.txt                   → Predicted class: 2
1_2.txt                   → Predicted class: 2
1_3.txt                   → Predicted class: 2
1_4.txt                   → Predicted class: 2
1_5.txt                   → Predicted class: 2
2_1.txt                   → Predicted class: 2
2_2.txt                   → Predicted class: 2
2_3.txt                   → Predicted class: 2
2_4.txt                   → Predicted class: 2
2_5.txt                   → Predicted class: 2
3_1.txt                   → Predicted class: 2
3_2.txt                   → Predicted class: 2
3_3.txt                   → Predicted class: 2
3_4.txt                   → Predicted class: 2
3_5.txt                   → Predicted class: 2
Predicted class: 2


In [38]:
testdata = [0.35, 0.45, 0.60, 0.85, 0.25, 0.90, 0.30, 0.75]
print("Returned Class: ")
print(knn_classifier_a.predict([testdata]))

Returned Class: 
[1]
